# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

[18, 17]


,aware,danger,elevation,feature,key,nation,nation number,rainfall,terrain,turn_last_visited,visited,x,y
area,0,0.132,9,Earthmark,18:17,Brinehaven,3,0,town,0,1,18,17
NArea,0,1.149,15,none,18:16,Brinehaven,3,0,desert,0,0,18,16
SArea,1,0.829,5,none,18:18,none,none,0,ocean,0,0,18,18
EArea,0,0.842,9,none,19:17,Brinehaven,3,0,desert,0,0,19,17
WArea,0,0.661,8,none,17:17,Brinehaven,3,0,desert,0,0,17,17


For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [5]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

True

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [6]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,0.0,0,0:0,16.0,desert,NaN,2.0,Landsfellow,0,0,0,0.573
1:0,0,0.0,1,1:0,20.0,desert,NaN,2.0,Landsfellow,0,0,0,0.682
2:0,0,0.0,2,2:0,19.0,desert,NaN,2.0,Landsfellow,0,0,0,0.811
5:0,0,0.0,5,5:0,14.0,desert,NaN,2.0,Landsfellow,0,0,0,0.733
6:0,0,0.0,6,6:0,12.0,desert,NaN,2.0,Landsfellow,0,0,0,1.462


You can also _mask_ the terrain where the character has no knowledge. 

In [7]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,0.0,0,0:0,16.0,unknown,unknown,0,unknown,0,0,0,0.573
1:0,0,0.0,1,1:0,20.0,unknown,unknown,0,unknown,0,0,0,0.682
2:0,0,0.0,2,2:0,19.0,unknown,unknown,0,unknown,0,0,0,0.811
3:0,0,0.0,3,3:0,18.0,unknown,unknown,0,unknown,0,0,0,0.171
4:0,0,0.0,4,4:0,17.0,unknown,unknown,0,unknown,0,0,0,0.333


Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [8]:
[[person.name for person in t.population] for t in world.towns]

[['Deldec', 'Frandun', 'Decknan'],
 ['Quardich', 'Marandos', 'Jogrox', 'Rojennunt'],
 ['Mantove', 'Brokelless', 'Latendod', 'Noyangox', 'Scofallesh'],
 ['Urfallod', 'Sereshol', 'Brolordale'],
 ['Tringan', 'Townan'],
 ['Frachust', 'Bafaddin', 'Neppsag', 'Darposh', 'Quaklak'],
 ['Zhaplent', 'Drawag', 'Wratallea', 'Alflak', 'Seblelle'],
 ['Hacinnec', 'Fentale', 'Scofloe', 'Lowillith', 'Lakrenton'],
 ['Afless', 'Secarraf', 'Vawildard'],
 ['Reyangess', 'Stidsah', 'Lahane'],
 ['Repprar', 'Erowildut'],
 ['Shensith', 'Vogennart'],
 ['Elossadle', 'Zhonnak'],
 ['Chickir', 'Loboldaker'],
 ['Lozennec'],
 ['Mefarrive', 'Zhopless'],
 ['Berpic', 'Kilarkoch'],
 ['Zhalarkane'],
 ['Erespnan']]

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [9]:
[[(person.name,person.temperment) for person in t.population if person.temperment>.5] for t in world.towns]

[[('Frandun', 0.63)],
 [('Quardich', 0.54), ('Marandos', 0.91), ('Rojennunt', 0.76)],
 [('Mantove', 0.67), ('Latendod', 0.74)],
 [('Urfallod', 0.64), ('Sereshol', 0.83), ('Brolordale', 0.8)],
 [('Townan', 0.55)],
 [('Neppsag', 0.64), ('Darposh', 0.78)],
 [('Drawag', 0.78)],
 [('Lakrenton', 0.97)],
 [('Vawildard', 0.78)],
 [('Stidsah', 0.82)],
 [],
 [],
 [('Elossadle', 0.52)],
 [],
 [('Lozennec', 0.68)],
 [('Mefarrive', 0.6)],
 [('Berpic', 0.71)],
 [('Zhalarkane', 0.55)],
 [('Erespnan', 0.6)]]

You can also get summary stats on people. For example, the average temperment per town.

In [10]:
[(t.name,np.mean([person.temperment for person in t.population])) for t in world.towns]

[('Founderstear', 0.4033333333333333),
 ('Gleamhost', 0.5900000000000001),
 ('Stormpike', 0.386),
 ('Splittown', 0.7566666666666667),
 ('Flameglen', 0.29000000000000004),
 ('Buckmelt', 0.35200000000000004),
 ('Warcatch', 0.4),
 ('Magering', 0.358),
 ('Soilwish', 0.43333333333333335),
 ('Doomchild', 0.47333333333333333),
 ('Magechild', 0.32),
 ('Earthmark', 0.33999999999999997),
 ('Lordsjaw', 0.28),
 ('Badgermark', 0.32499999999999996),
 ('Earthhenge', 0.68),
 ('Lordsring', 0.42),
 ('Soilchild', 0.52),
 ('Flamerise', 0.55),
 ('Lionfort', 0.6)]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [11]:
where_the_char_has_been = world.df_features.loc[world.df_features['visited']==1]
where_the_char_has_been

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
18:17,17,0.0,18,18:17,9.0,town,Earthmark,3.0,Brinehaven,1,0,0,0.132


In [12]:
where_the_char_has_been['nation'].dropna()

key
18:17    Brinehaven
Name: nation, dtype: object

In [13]:
nations_where_character_has_been = np.unique(where_the_char_has_been['nation'].dropna())
nations_where_character_has_been

array(['Brinehaven'], dtype=object)

You can use this to get lists of people who the character knows. 

In [14]:
[[t for t in n.towns] for n in world.nations if n.name in nations_where_character_has_been]

[[capitol of Gleamhost: population: 3 location: [24,16] founded 0,
  town of Flameglen: population: 2 location: [20,13] founded 3,
  town of Earthmark: population: 2 location: [18,17] founded 6]]

In [15]:
[[t for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

[[Quardich the Speaker of Gleamhost,
  Marandos the commoner,
  Jogrox the commoner,
  Rojennunt the Ruler of the nation of Brinehaven],
 [Tringan the Speaker of Flameglen, Townan the commoner],
 [Shensith the Speaker of Earthmark, Vogennart the commoner]]

In [16]:
[[t.get_person_data() for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

[[{'name': 'Quardich',
   'role': 'Speaker of Gleamhost',
   'loyalty': 0.5,
   'temperment': 0.54,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'What a great wedding. So glad the people of [Nation of Brinehaven] are our friends.']},
  {'name': 'Marandos',
   'role': 'commoner',
   'loyalty': 0.5,
   'temperment': 0.91,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'What a great wedding. So glad the people of [Nation of Brinehaven] are our friends.']},
  {'name': 'Jogrox',
   'role': 'commoner',
   'loyalty': 0.5,
   'temperment': 0.15,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'What a great wedding. So glad the people of [Nation of Brinehaven] are our friends.']},
  {'name': 'Rojennunt',
   'role': 'Ruler of the nation of Brinehaven',
   'loyalty': 0.5,
   'temperment': 0.76,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'What a great wedding. So glad the people of [Nation of Brinehaven] are our f

In [17]:
masked_world.loc[world.Character.get_location_key()]

y                            17
rainfall                      0
x                            18
key                       18:17
elevation                     9
terrain                    town
feature               Earthmark
nation number                 3
nation               Brinehaven
visited                       1
aware                         0
turn_last_visited             0
danger                    0.132
Name: 18:17, dtype: object

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [18]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
18:17,17,0.0,18,18:17,9.0,town,Earthmark,3.0,Brinehaven,1,0,0,0.132


## Getting the diplomacy and relationships data for the journal page

In [19]:
rel = w.get_people_where_char_has_visited(world)
rel

[{'town': town of Earthmark: population: 2 location: [18,17] founded 6,
  'people': [Shensith the Speaker of Earthmark, Vogennart the commoner]}]

In [20]:
w.get_relationships_node_map(world)

[{'name': 'Earthmark',
  'title': 'town of Earthmark',
  'nation': 'Brinehaven',
  'population': 2,
  'type': 'town',
  'location': '18:17',
  'founded year': 6,
  'national fealty': 1,
  'children': [{'name': 'Shensith',
    'role': 'Speaker of Earthmark',
    'temperment': 0.43,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Vogennart',
    'role': 'commoner',
    'temperment': 0.25,
    'loyalty': 0.5,
    'type': 'person'}]}]

In [21]:
hierarchy = [{'nation':i.name,
              'children':i.diplomacy} for i in world.nations]



In [22]:
p = rel[0]['town'].population[0]


In [26]:
def get_people_where_char_has_visited(world):
    """
    returns a dictionary of {{"town":[<obj>],"people":[<obj>]}
    filtered to where the character has been. 
    """
    where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                    (world.df_features['terrain']=='town')].dropna()
    towns_and_people = [{"town":T,
                         "people":[t for t in T.population]} 
                        for T in world.towns if T.name in np.unique(w.where_the_char_has_been['feature'])]
    return towns_and_people

def get_relationships_node_map(world):
    nodes = [{'name':r['town'].name,
              'title':str(r['town']).split(":")[0],
              'nation':r['town'].diplomacy['nation'],
              'population':r['town'].pop,
              'type':r['town'].type,
              'location':r['town'].key,
              'founded year':r['town'].founded,
              'national fealty':r['town'].diplomacy['national fealty'],
             'children': [{'name':p.name,
                          'role':p.role,
                          'temperment':p.temperment,
                          'loyalty':p.loyalty,
                          'type':'person'} for p in r['people']]} 
             for r in w.get_people_where_char_has_visited(world)]
    return nodes
        
w.get_people_where_char_has_visited(world)

[{'town': town of Earthmark: population: 2 location: [18,17] founded 6,
  'people': [Shensith the Speaker of Earthmark, Vogennart the commoner]}]

In [28]:
relationships = w.get_relationships_node_map(world)
relationships

[{'name': 'Earthmark',
  'title': 'town of Earthmark',
  'nation': 'Brinehaven',
  'population': 2,
  'type': 'town',
  'location': '18:17',
  'founded year': 6,
  'national fealty': 1,
  'children': [{'name': 'Shensith',
    'role': 'Speaker of Earthmark',
    'temperment': 0.43,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Vogennart',
    'role': 'commoner',
    'temperment': 0.25,
    'loyalty': 0.5,
    'type': 'person'}]}]

In [36]:
nations = [s['nation'] for s in relationships]
nations

['Brinehaven']

In [40]:
[i for i in relationships]

[{'name': 'Earthmark',
  'title': 'town of Earthmark',
  'nation': 'Brinehaven',
  'population': 2,
  'type': 'town',
  'location': '18:17',
  'founded year': 6,
  'national fealty': 1,
  'children': [{'name': 'Shensith',
    'role': 'Speaker of Earthmark',
    'temperment': 0.43,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Vogennart',
    'role': 'commoner',
    'temperment': 0.25,
    'loyalty': 0.5,
    'type': 'person'}]}]

In [49]:
[{'nation':i,'towns':[t for t in relationships if t['nation']==i]} for i in nations]

[{'nation': 'Brinehaven',
  'towns': [{'name': 'Earthmark',
    'title': 'town of Earthmark',
    'nation': 'Brinehaven',
    'population': 2,
    'type': 'town',
    'location': '18:17',
    'founded year': 6,
    'national fealty': 1,
    'children': [{'name': 'Shensith',
      'role': 'Speaker of Earthmark',
      'temperment': 0.43,
      'loyalty': 0.5,
      'type': 'person'},
     {'name': 'Vogennart',
      'role': 'commoner',
      'temperment': 0.25,
      'loyalty': 0.5,
      'type': 'person'}]}]}]

In [46]:
[i for i in nations]

['Brinehaven']